# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [2]:
import requests
from bs4 import BeautifulSoup

def soupify(url):
    return BeautifulSoup(requests.get(url).text, 'html.parser')

In [4]:
tf_api = {}
for a in soupify("https://www.tensorflow.org/api_docs/python/").find_all('a', href=True):
    if 'api_docs/python/tf' in a['href']:
        tf_api[a.text] = soupify(a['href']).find('article', attrs={'class':'devsite-article-inner'}).text

print(tf_api)

KeyboardInterrupt: 

In [ ]:
mx_api = {}
MX_BASE = "https://mxnet.incubator.apache.org/api/python/"
for a in soupify(MX_BASE + "index.html").find_all('a', href=True, attrs={"class":"reference internal"}):
    if '..' not in a['href'] and a['href'] not in mx_api:
        for dl in soupify(MX_BASE + a['href']).find_all('dl', attrs={'class':'method'}):
            try:
                mx_api[dl.find('dt')['id']] = dl.text
            except:
                pass

print(mx_api)